<a href="https://colab.research.google.com/github/magdalena-b/Bairon/blob/master/DualRL_Shakespeare_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install OpenNMT-tf==1.15.0

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
%env DATASET=yelp

env: DATASET=yelp


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/DualRL

/content/drive/MyDrive/DualRL


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/DualRL')

In [ ]:
import utils

In [ ]:
!ls

classifier	   dual_options.py   LICENSE  __pycache__  tmp
common_options.py  dual_training.py  nmt      README.md    utils
data		   fig		     outputs  references


In [ ]:
import classifier.options

In [ ]:
from classifier.model import TextCNN

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
!pip install easydict

In [ ]:
import tensorflow as tf
import numpy as np
import random
import re
import time
import sys
sys.path.append('..')
from classifier.model import TextCNN
from classifier.options import *
from utils import constants
from utils.vocab import build_vocab_from_file, load_vocab_dict
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2,1,0,3"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


def create_model(sess, args, vocab_size, mode=constants.TRAIN, load_pretrained_model=False, reuse=None):
    with tf.variable_scope(constants.CLS_VAR_SCOPE, reuse=reuse):
        model = TextCNN(mode, args.__dict__, vocab_size)

    if load_pretrained_model:
        try:
            model.saver.restore(sess, args.cls_model_save_dir)
            print("Loading model from", args.cls_model_save_dir)
        except Exception as e:
            model.saver.restore(sess, tf.train.latest_checkpoint(args.cls_model_save_dir))
            print("Loading model from", tf.train.latest_checkpoint(args.cls_model_save_dir))
    else:
        if reuse is None:
            print("Creating model with new parameters.")
            sess.run(tf.global_variables_initializer())
        else:
            print('Reuse parameters.')
    return model


def evaluate(sess, args, vocab, model, x, y, print_logs=True):
    probs = []
    batches = get_batches(x, y, word2id=vocab, batch_size=1)
    for batch in batches:
        p = sess.run(model.probs,
                     feed_dict={model.x: batch["x"],
                                model.dropout: 1})
        probs += p.tolist()
    y_hat = [p > 0.5 for p in probs]
    same = [p == q for p, q in zip(y, y_hat)]

    if print_logs:
        print("Saving classifier result at: %s" % args.log_path)
    with open(args.log_path, 'w') as f:
        for i in range(len(y)):
            f.write("%s\t%.3f\t%s\n" % (' '.join(x[i]), probs[i], same[i]))
    return (100.0 * sum(same)) / len(y), probs


def get_batches(x, y, word2id, batch_size, min_len=5):
    pad = word2id[constants.PADDING_TOKEN]
    unk = word2id[constants.UNKNOWN_TOKEN]

    batches = []
    s = 0
    sen_len = []
    while s < len(x):
        t = min(s + batch_size, len(x))

        _x = []
        max_len = max([len(sent) for sent in x[s:t]])
        max_len = max(max_len, min_len)  # sensitive to sentence-length
        sen_len.append(max_len)
        for sent in x[s:t]:
            sent_id = [word2id[w] if w in word2id else unk for w in sent]
            padding = [pad] * (max_len - len(sent))
            _x.append(padding + sent_id)

        batches.append({"x": _x,
                        "y": y[s:t]})
        s = t
    return batches


def prepare(paths, index_list=None, is_training=False):
    def load_sent(path, max_size=-1):
        data = []
        with open(path) as f:
            for line in f:
                if len(data) == max_size:
                    break
                words = line.split()
                if is_training:
                    if len(words) > 1:  # filter sentence only have one words
                        data.append(words)
                else:
                    data.append(words)
        return data

    if index_list is None:
        index_list = []
        for path in paths:
            i = int(re.findall('\d', path)[-1])
            if '.tsf' in path or 'reference' in path:
                i = 1-i
            index_list.append(i)

    data0 = load_sent(paths[0])
    if len(paths) >= 2:
        data1 = load_sent(paths[1])
        if is_training:
            min_c = min(len(data0), len(data1))
            np.random.shuffle(data0)
            np.random.shuffle(data1)
            # balance the bias caused by training data
            print("Dropped: %d, %s" % (len(data0) - min_c, paths[0]))
            data0 = data0[:min_c]
            print("Dropped: %d, %s" % (len(data1) - min_c, paths[1]))
            data1 = data1[:min_c]

        x = data0 + data1
        y = [index_list[0]] * len(data0) + [index_list[1]] * len(data1)
    else:
        x = data0
        y = [index_list[0]] * len(data0)

    z = sorted(zip(x, y), key=lambda i: len(i[0]))  # ranked by the length of sentences
    return zip(*z)


def evaluate_file(sess, args, vocab, eval_model, files, index_list, print_logs=True):
    x, y = prepare(files, index_list=index_list)
    acc, _ = evaluate(sess, args, vocab, eval_model, x, y, print_logs)
    return acc


if __name__ == "__main__":
    args = load_cls_arguments()

    if args.train_data and args.mode == constants.TRAIN:
        train_x, train_y = prepare(args.train_data, index_list=[0, 1], is_training=True)

        if not os.path.isfile(args.global_vocab_file):
            build_vocab_from_file(args.train_data, args.global_vocab_file)

    vocab, vocab_size = load_vocab_dict(args.global_vocab_file)
    print("Vocabulary size", vocab_size)

    if args.dev_data:
        dev_x, dev_y = prepare(args.dev_data)

    # if args.test_data and args.mode == constants.EVAL:
    if args.test_data:
        test_x, test_y = prepare(args.test_data)

    print("Prepare to Save model at: %s" % args.cls_model_save_dir)
    if not os.path.exists(args.cls_model_save_dir):
        os.makedirs(args.cls_model_save_dir)

    dump_args_to_yaml(args, args.cls_model_save_dir)

    config = tf.ConfigProto()
    # config.gpu_options.allow_growth = True
    # config.gpu_options.per_process_gpu_memory_fraction = 0.2
    with tf.Session(config=config) as sess:
        with tf.device("/cpu:0"):   # gpu will face error
            if args.mode == constants.TRAIN:
                model = create_model(sess, args, vocab_size)
                if args.train_data:
                    batches = get_batches(train_x, train_y,
                                          vocab, args.batch_size)
                    random.shuffle(batches)

                    start_time = time.time()
                    step = 0
                    loss = 0.0
                    best_dev = float("-inf")
                    learning_rate = args.learning_rate

                    for epoch in range(1, 1 + args.n_epoch):
                        print("--------------------Epoch %d--------------------" % epoch)

                        for batch in batches:
                            step_loss, _ = sess.run([model.loss, model.optimizer],
                                                    feed_dict={model.x: batch["x"],
                                                               model.y: batch["y"],
                                                               model.dropout: args.keep_prob})

                            step += 1
                            loss += step_loss / args.steps_per_checkpoint

                            if step % args.steps_per_checkpoint == 0:
                                print("step %d, time %.0fs, loss %.2f" % (step, time.time() - start_time, loss))
                                loss = 0.0

                        if args.dev_data:
                            acc, _ = evaluate(sess, args, vocab, model, dev_x, dev_y)
                            print("Dev accuracy: %.2f" % acc)
                            if acc > best_dev:
                                best_dev = acc
                                print("Saving model to:  %s" % args.cls_model_save_dir)
                                model.saver.save(sess, args.cls_model_save_dir)
                        if args.test_data:
                            acc, _ = evaluate(sess, args, vocab, model, test_x, test_y)
                            print("Test accuracy: %.2f" % acc)
            else:
                if args.test_data:
                    eval_model = create_model(sess, args, vocab_size, load_pretrained_model=True)
                    acc, _ = evaluate(sess, args, vocab, eval_model, test_x, test_y)
                    print("Test accuracy: %.2f" % acc)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Dropped: 0, /content/drive/MyDrive/DualRL/data/yelp/train.0
Dropped: 88887, /content/drive/MyDrive/DualRL/data/yelp/train.1
Vocabulary size 9353
Prepare to Save model at: /content/drive/MyDrive/DualRL/tmp/model/yelp/cls/
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Creating model with new parameters.
--------------------Epoch 1--------------------
step 100, time 5s, loss 0.65
step 200, time 10s, loss 0.28
step 300, time 15s, loss 0.22
step 400, time 20s, loss 0.22
step 500, time 25s, loss 0.

https://newbedev.com/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter


https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

https://github.com/makinacorpus/easydict/blob/master/easydict/__init__.py